# Test Kaggle Directory

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/vs-brats2018/miccai_brats_2018_data_training"))
print(os.listdir("/kaggle"))

# Any results you write to the current directory are saved as output.

In [ ]:
ls /kaggle/input

# Delete Directories(Optional)
*   For deleing the generated database.

In [ ]:
if (True):
    import os
    import sys
    import shutil

    try:
        shutil.rmtree("../BraTs_Training_p508_npy/")
    except OSError as e:
        print ("Error: %s - %s." % (e.filename, e.strerror))

# Pre-processing

In [ ]:
import os
import sys
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from time import time

IMG_WIDTH = 240
IMG_HEIGHT = 240
IMG_CHANNELS = 4
HW = 100

enable_original_axial = True;
enable_original_sagittal = True;
enable_original_coronal = True;

TRAIN_PATH = "../input/vs-brats2018/miccai_brats_2018_data_training"

AXIAL_DATA_PATH = "../BraTs_Training_p508_npy/original/axial/Data"
AXIAL_LABEL_PATH = "../BraTs_Training_p508_npy/original/axial/Label"

SAGITTAL_DATA_PATH = "../BraTs_Training_p508_npy/original/sagittal/Data"
SAGITTAL_LABEL_PATH = "../BraTs_Training_p508_npy/original/sagittal/Label"

CORONAL_DATA_PATH = "../BraTs_Training_p508_npy/original/coronal/Data"
CORONAL_LABEL_PATH = "../BraTs_Training_p508_npy/original/coronal/Label"


import matplotlib.image as mpimg
from scipy.stats import skew
from scipy.stats import kurtosis
import numpy.ma as ma

dirs_all = os.listdir(TRAIN_PATH)
dirs_all.sort()
dirs_train = dirs_all[:2]
training_data = []
label_data = []


def normalize_image(im):
  im_mask = (im > 0);
  im_compressed_region = ma.array(im,mask=~im_mask).compressed()
  #print(im_compressed_region.size)
  
  #plt.imshow(im_mask)
  #plt.show()
  #plt.imshow(im)
  #plt.show()
  if (im_compressed_region.size == 0):
    return im
  else:
    avg = np.mean(im_compressed_region,dtype='float32')
    std = np.std(im_compressed_region,dtype='float32')
    return (ma.array(im,mask=~im_mask)-avg)/std

axial_data_path = [];
axial_label_path = [];

sagittal_data_path = [];
sagittal_label_path = [];

coronal_data_path = [];
coronal_label_path = [];
for nf, folder in enumerate(dirs_train):
  print("Processing Images...")
  HLG_PATH = os.path.join(TRAIN_PATH,folder)
  hlg_all = os.listdir(HLG_PATH)
  for n, file in enumerate(hlg_all):
    print("Image: "+ str(nf) + ", " + str(n))
    flair = nib.load(os.path.join(HLG_PATH, file, file + '_flair.nii.gz')).get_data()
    t1 = nib.load(os.path.join(HLG_PATH, file, file + '_t1.nii.gz')).get_data()
    t1ce = nib.load(os.path.join(HLG_PATH, file, file + '_t1ce.nii.gz')).get_data()
    t2 = nib.load(os.path.join(HLG_PATH, file, file + '_t2.nii.gz')).get_data()
    label = nib.load(os.path.join(HLG_PATH, file, file + '_seg.nii.gz')).get_data()
    
    if(enable_original_axial):
        print("Axial Data:")    
        folder_directory = os.path.join(AXIAL_DATA_PATH,folder,file)
        if not os.path.exists(folder_directory):
          os.makedirs(folder_directory)
          for i in range(155):
            if(np.mean(flair[:,:,i]+t1[:,:,i]+t1ce[:,:,i]+t2[:,:,i])!=0):
                data_patch_axial = np.zeros([240,240,4],dtype='float32')

                data_patch_axial[:,:,0] = normalize_image(flair[:,:,i])
                data_patch_axial[:,:,1] = normalize_image(t1[:,:,i])
                data_patch_axial[:,:,2] = normalize_image(t1ce[:,:,i])
                data_patch_axial[:,:,3] = normalize_image(t2[:,:,i])

                np.savez_compressed(os.path.join(folder_directory,'pao'+ '_' + str(nf) + '_' + str(n) + '_' + str(i) + '.npz'),data_patch_axial)
                axial_data_path.append([nf,n,os.path.join(folder_directory,'pao'+ '_' + str(nf) + '_' + str(n) + '_' + str(i) + '.npz')]);  
                

        print("Axial Label:")
        folder_directory = os.path.join(AXIAL_LABEL_PATH,folder,file)
        if not os.path.exists(folder_directory):
          os.makedirs(folder_directory)

          for i in range(155):
            if(np.mean(flair[:,:,i]+t1[:,:,i]+t1ce[:,:,i]+t2[:,:,i])!=0):
                label_patch_axial = np.zeros([240,240,4],dtype='int')

                label_patch_axial[:,:,0] = (label[:,:,i]==0).astype(int)
                label_patch_axial[:,:,1] = (label[:,:,i]==1).astype(int)
                label_patch_axial[:,:,2] = (label[:,:,i]==2).astype(int)
                label_patch_axial[:,:,3] = (label[:,:,i]==4).astype(int)

                np.savez_compressed(os.path.join(folder_directory,'pao'+ '_' + str(nf) + '_' + str(n) + '_' + str(i) + '.npz'),label_patch_axial)
                axial_label_path.append([nf,n,os.path.join(folder_directory,'pao'+ '_' + str(nf) + '_' + str(n) + '_' + str(i) + '.npz')]);

    ##############################################################################################################################################################################################
    if(enable_original_sagittal):
        print("Sagittal Data:")
        folder_directory = os.path.join(SAGITTAL_DATA_PATH,folder,file)
        if not os.path.exists(folder_directory):
          os.makedirs(folder_directory)
          for i in range(240):
            if(np.mean(flair[i,:,:]+t1[i,:,:]+t1ce[i,:,:]+t2[i,:,:])!=0):
                data_patch_sagittal = np.zeros([240,240,4],dtype='float32')

                data_patch_sagittal[:,41:196,0] = normalize_image(flair[i,:,:])
                data_patch_sagittal[:,41:196,1] = normalize_image(t1[i,:,:])
                data_patch_sagittal[:,41:196,2] = normalize_image(t1ce[i,:,:])
                data_patch_sagittal[:,41:196,3] = normalize_image(t2[i,:,:])

                np.savez_compressed(os.path.join(folder_directory,'pso'+ '_' + str(nf) + '_' + str(n) + '_' + str(i) + '.npz'),data_patch_sagittal)
                sagittal_data_path.append([nf,n,os.path.join(folder_directory,'pso'+ '_' + str(nf) + '_' + str(n) + '_' + str(i) + '.npz')]);

        print("Sagittal Label:")
        folder_directory = os.path.join(SAGITTAL_LABEL_PATH,folder,file)
        if not os.path.exists(folder_directory):
          os.makedirs(folder_directory)
          for i in range(240):
            if(np.mean(flair[i,:,:]+t1[i,:,:]+t1ce[i,:,:]+t2[i,:,:])!=0):
                label_patch_sagittal = np.zeros([240,240,4],dtype='int')
                
                label_patch_sagittal[:,:,0] = label_patch_sagittal[:,:,0] + 1
                label_patch_sagittal[:,41:196,0] = (label[i,:,:]==0).astype(int)
                label_patch_sagittal[:,41:196,1] = (label[i,:,:]==1).astype(int)
                label_patch_sagittal[:,41:196,2] = (label[i,:,:]==2).astype(int)
                label_patch_sagittal[:,41:196,3] = (label[i,:,:]==4).astype(int)
                
                np.savez_compressed(os.path.join(folder_directory,'pso'+ '_' + str(nf) + '_' + str(n) + '_' + str(i) + '.npz'),label_patch_sagittal)
                sagittal_label_path.append([nf,n,os.path.join(folder_directory,'pso'+ '_' + str(nf) + '_' + str(n) + '_' + str(i) + '.npz')]);

    ##############################################################################################################################################################################################
    if(enable_original_coronal):
        print("Coronal Data:")
        folder_directory = os.path.join(CORONAL_DATA_PATH,folder,file)
        if not os.path.exists(folder_directory):
          os.makedirs(folder_directory)
          for i in range(240):
            if(np.mean(flair[:,i,:]+t1[:,i,:]+t1ce[:,i,:]+t2[:,i,:])!=0):
                data_patch_coronal = np.zeros([240,240,4],dtype='float32')

                data_patch_coronal[:,41:196,0] = normalize_image(flair[:,i,:])
                data_patch_coronal[:,41:196,1] = normalize_image(t1[:,i,:])
                data_patch_coronal[:,41:196,2] = normalize_image(t1ce[:,i,:])
                data_patch_coronal[:,41:196,3] = normalize_image(t2[:,i,:])

                np.savez_compressed(os.path.join(folder_directory,'pco'+ '_' + str(nf) + '_' + str(n) + '_' + str(i) + '.npz'),data_patch_coronal)
                coronal_data_path.append([nf,n,os.path.join(folder_directory,'pco'+ '_' + str(nf) + '_' + str(n) + '_' + str(i) + '.npz')]);

        print("Coronal Label:")          
        folder_directory = os.path.join(CORONAL_LABEL_PATH,folder,file)
        if not os.path.exists(folder_directory):
          os.makedirs(folder_directory)
          for i in range(240):
            if(np.mean(flair[:,i,:]+t1[:,i,:]+t1ce[:,i,:]+t2[:,i,:])!=0):
                label_patch_coronal = np.zeros([240,240,4],dtype='int')
                
                label_patch_coronal[:,:,0] = label_patch_coronal[:,:,0] + 1
                label_patch_coronal[:,41:196,0] = (label[:,i,:]==0).astype(int)
                label_patch_coronal[:,41:196,1] = (label[:,i,:]==1).astype(int)
                label_patch_coronal[:,41:196,2] = (label[:,i,:]==2).astype(int)
                label_patch_coronal[:,41:196,3] = (label[:,i,:]==4).astype(int)

                np.savez_compressed(os.path.join(folder_directory,'pco'+ '_' + str(nf) + '_' + str(n) + '_' + str(i) + '.npz'),label_patch_coronal)
                coronal_label_path.append([nf,n,os.path.join(folder_directory,'pco'+ '_' + str(nf) + '_' + str(n) + '_' + str(i) + '.npz')]);

if(enable_original_axial):
    np.savez_compressed(os.path.join('../BraTs_Training_p508_npy/axial_data_path'+ '.npz'),axial_data_path)    
    np.savez_compressed(os.path.join('../BraTs_Training_p508_npy/axial_label_path'+ '.npz'),axial_label_path)
if(enable_original_sagittal):
    np.savez_compressed(os.path.join('../BraTs_Training_p508_npy/sagittal_data_path'+ '.npz'),sagittal_data_path)    
    np.savez_compressed(os.path.join('../BraTs_Training_p508_npy/sagittal_label_path'+ '.npz'),sagittal_label_path)
if(enable_original_coronal):
    np.savez_compressed(os.path.join('../BraTs_Training_p508_npy/coronal_data_path'+ '.npz'),coronal_data_path)    
    np.savez_compressed(os.path.join('../BraTs_Training_p508_npy/coronal_label_path'+ '.npz'),coronal_label_path)

# Show generated numpy data

In [ ]:
#del axial_data_path
#del axial_label_path
#del data_patch_axial
#del label_patch_axial

In [ ]:
#for paient_idx in range(len(dirs_axial_data_decompress)):
#    if (dirs_axial_data_decompress[paient_idx][2].find('Brats18_TCIA10_266_1') != -1): 
#        print (dirs_axial_data_decompress[paient_idx][0])
#        print (dirs_axial_data_decompress[paient_idx][1])
    #if(dirs_axial_data_decompress[paient_idx][0] == str(0)):
        #if(dirs_axial_data_decompress[paient_idx][1] == str(5)):
            #print(dirs_axial_data_decompress[paient_idx][2])
#print(dirs_axial_data_decompress[0][0])
#print(dirs_axial_data_decompress[0][1])
#print(dirs_axial_data_decompress[0][2])


import random

Testing_patients_index = [[0,36],[0,128],[0,68],[0,17],[0,32],[0,180],[0,54],[1,0],[1,7],[1,57]]

LGG_testing_idx  = [0,7,57]
HGG_testing_idx  = [36,128,68,17,32,180,54]

print("Start: Testing")
print(len(LGG_testing_idx))
print(len(HGG_testing_idx))
LGG_non_testing_idx = [x for x in range(0,75) if x not in LGG_testing_idx]
HGG_non_testing_idx = [x for x in range(0,210) if x not in HGG_testing_idx]

LGG_validation_idx = LGG_non_testing_idx[:8]
HGG_validation_idx = HGG_non_testing_idx[:22]

print("Start: Validation")
print(len(LGG_validation_idx))
print(len(HGG_validation_idx))

LGG_training_idx = LGG_non_testing_idx[8:]
HGG_training_idx = HGG_non_testing_idx[22:]
print("Start: Training")
print(len(LGG_training_idx))
print(len(HGG_training_idx))


if(enable_original_axial):
    dirs_axial_data = np.load('../BraTs_Training_p508_npy/axial_data_path'+ '.npz')
    dirs_axial_label = np.load('../BraTs_Training_p508_npy/axial_label_path'+ '.npz')
    dirs_axial_data_decompress = dirs_axial_data['arr_0']
    dirs_axial_label_decompress = dirs_axial_label['arr_0']
    dirs_axial_data_training = [];
    dirs_axial_label_training = [];
    dirs_axial_data_validation = [];
    dirs_axial_label_validation = [];
    dirs_axial_data_testing = [];
    dirs_axial_label_testing = [];
    hgg_len = 0
    lgg_len =0
    hgg_len_val = 0
    lgg_len_val =0
    hgg_len_train = 0
    lgg_len_train =0
    hgg_len_test = 0
    lgg_len_test =0
    
    for dirs_idx in range(len(dirs_axial_data_decompress)):
        if(dirs_axial_data_decompress[dirs_idx][0] == str(0)):
            #hgg_len = hgg_len+1
            for idx in range(len(HGG_testing_idx)):
                if(dirs_axial_data_decompress[dirs_idx][1]==str(HGG_testing_idx[idx])):
                    #hgg_len_test = hgg_len_test + 1
                    #print(dirs_axial_data_decompress[dirs_idx][1])
                    dirs_axial_data_testing.append(dirs_axial_data_decompress[dirs_idx][2])
                    dirs_axial_label_testing.append(dirs_axial_label_decompress[dirs_idx][2])
                    
            for idx in range(len(HGG_validation_idx)):
                if(dirs_axial_data_decompress[dirs_idx][1]==str(HGG_validation_idx[idx])):
                    #hgg_len_val = hgg_len_val + 1
                    dirs_axial_data_validation.append(dirs_axial_data_decompress[dirs_idx][2])
                    dirs_axial_label_validation.append(dirs_axial_label_decompress[dirs_idx][2])
                    
            for idx in range(len(HGG_training_idx)):
                if(dirs_axial_data_decompress[dirs_idx][1]==str(HGG_training_idx[idx])):
                    #hgg_len_train = hgg_len_train + 1
                    dirs_axial_data_training.append(dirs_axial_data_decompress[dirs_idx][2])
                    dirs_axial_label_training.append(dirs_axial_label_decompress[dirs_idx][2])
            
        if(dirs_axial_data_decompress[dirs_idx][0] == str(1)):
            #lgg_len = lgg_len+1
            for idx in range(len(LGG_testing_idx)):
                if(dirs_axial_data_decompress[dirs_idx][1]==str(LGG_testing_idx[idx])):
                    #lgg_len_test = lgg_len_test + 1
                    dirs_axial_data_testing.append(dirs_axial_data_decompress[dirs_idx][2])
                    dirs_axial_label_testing.append(dirs_axial_label_decompress[dirs_idx][2])
                   
            for idx in range(len(LGG_validation_idx)):
                if(dirs_axial_data_decompress[dirs_idx][1]==str(LGG_validation_idx[idx])):
                    #lgg_len_val = lgg_len_val + 1
                    dirs_axial_data_validation.append(dirs_axial_data_decompress[dirs_idx][2])
                    dirs_axial_label_validation.append(dirs_axial_label_decompress[dirs_idx][2])
                   
            for idx in range(len(LGG_training_idx)):
                if(dirs_axial_data_decompress[dirs_idx][1]==str(LGG_training_idx[idx])):
                    #lgg_len_train = lgg_len_train + 1
                    dirs_axial_data_training.append(dirs_axial_data_decompress[dirs_idx][2])
                    dirs_axial_label_training.append(dirs_axial_label_decompress[dirs_idx][2])
    
    print(len(dirs_axial_data_decompress))
    print(len(dirs_axial_data_testing))
    print(len(dirs_axial_data_validation))
    print(len(dirs_axial_data_training))
    """
    print("HGG TOTAL")
    print(hgg_len)
    print(hgg_len_test)
    print(hgg_len_val)
    print(hgg_len_train)

    print("LGG TOTAL")
    print(lgg_len)
    print(lgg_len_test)
    print(lgg_len_val)
    print(lgg_len_train)
    """
if(enable_original_sagittal):
    dirs_sagittal_data = np.load('../BraTs_Training_p508_npy/sagittal_data_path'+ '.npz')
    dirs_sagittal_label = np.load('../BraTs_Training_p508_npy/sagittal_label_path'+ '.npz')
    dirs_sagittal_data_decompress = dirs_sagittal_data['arr_0']
    dirs_sagittal_label_decompress = dirs_sagittal_label['arr_0']
    dirs_sagittal_data_training = [];
    dirs_sagittal_label_training = [];
    dirs_sagittal_data_validation = [];
    dirs_sagittal_label_validation = [];
    dirs_sagittal_data_testing = [];
    dirs_sagittal_label_testing = [];
    
    for dirs_idx in range(len(dirs_sagittal_data_decompress)):

        if(dirs_sagittal_data_decompress[dirs_idx][0] == str(0)):
            for idx in range(len(HGG_testing_idx)):
                if(dirs_sagittal_data_decompress[dirs_idx][1]==str(HGG_testing_idx[idx])):
                    dirs_sagittal_data_testing.append(dirs_sagittal_data_decompress[dirs_idx][2])
                    dirs_sagittal_label_testing.append(dirs_sagittal_label_decompress[dirs_idx][2])

            for idx in range(len(HGG_validation_idx)):
                if(dirs_sagittal_data_decompress[dirs_idx][1]==str(HGG_validation_idx[idx])):
                    dirs_sagittal_data_validation.append(dirs_sagittal_data_decompress[dirs_idx][2])
                    dirs_sagittal_label_validation.append(dirs_sagittal_label_decompress[dirs_idx][2])

            for idx in range(len(HGG_training_idx)):
                if(dirs_sagittal_data_decompress[dirs_idx][1]==str(HGG_training_idx[idx])):
                    dirs_sagittal_data_training.append(dirs_sagittal_data_decompress[dirs_idx][2])
                    dirs_sagittal_label_training.append(dirs_sagittal_label_decompress[dirs_idx][2])

        if(dirs_sagittal_data_decompress[dirs_idx][0] == str(1)):
            for idx in range(len(LGG_testing_idx)):
                if(dirs_sagittal_data_decompress[dirs_idx][1]==str(LGG_testing_idx[idx])):
                    dirs_sagittal_data_testing.append(dirs_sagittal_data_decompress[dirs_idx][2])
                    dirs_sagittal_label_testing.append(dirs_sagittal_label_decompress[dirs_idx][2])

            for idx in range(len(LGG_validation_idx)):
                if(dirs_sagittal_data_decompress[dirs_idx][1]==str(LGG_validation_idx[idx])):
                    dirs_sagittal_data_validation.append(dirs_sagittal_data_decompress[dirs_idx][2])
                    dirs_sagittal_label_validation.append(dirs_sagittal_label_decompress[dirs_idx][2])

            for idx in range(len(LGG_training_idx)):
                if(dirs_sagittal_data_decompress[dirs_idx][1]==str(LGG_training_idx[idx])):
                    dirs_sagittal_data_training.append(dirs_sagittal_data_decompress[dirs_idx][2])
                    dirs_sagittal_label_training.append(dirs_sagittal_label_decompress[dirs_idx][2])
    
    print(len(dirs_sagittal_data_decompress))
    print(len(dirs_sagittal_data_testing))
    print(len(dirs_sagittal_data_validation))
    print(len(dirs_sagittal_data_training))
                    
if(enable_original_coronal):
    dirs_coronal_data = np.load('../BraTs_Training_p508_npy/coronal_data_path'+ '.npz')
    dirs_coronal_label = np.load('../BraTs_Training_p508_npy/coronal_label_path'+ '.npz')
    dirs_coronal_data_decompress = dirs_coronal_data['arr_0']
    dirs_coronal_label_decompress = dirs_coronal_label['arr_0']
    dirs_coronal_data_training = [];
    dirs_coronal_label_training = [];
    dirs_coronal_data_validation = [];
    dirs_coronal_label_validation = [];
    dirs_coronal_data_testing = [];
    dirs_coronal_label_testing = [];

    for dirs_idx in range(len(dirs_coronal_data_decompress)):
        if(dirs_coronal_data_decompress[dirs_idx][0] == str(0)):
            for idx in range(len(HGG_testing_idx)):
                if(dirs_coronal_data_decompress[dirs_idx][1]==str(HGG_testing_idx[idx])):
                    dirs_coronal_data_testing.append(dirs_coronal_data_decompress[dirs_idx][2])
                    dirs_coronal_label_testing.append(dirs_coronal_label_decompress[dirs_idx][2])

            for idx in range(len(HGG_validation_idx)):
                if(dirs_coronal_data_decompress[dirs_idx][1]==str(HGG_validation_idx[idx])):
                    dirs_coronal_data_validation.append(dirs_coronal_data_decompress[dirs_idx][2])
                    dirs_coronal_label_validation.append(dirs_coronal_label_decompress[dirs_idx][2])

            for idx in range(len(HGG_training_idx)):
                if(dirs_coronal_data_decompress[dirs_idx][1]==str(HGG_training_idx[idx])):
                    dirs_coronal_data_training.append(dirs_coronal_data_decompress[dirs_idx][2])
                    dirs_coronal_label_training.append(dirs_coronal_label_decompress[dirs_idx][2])

        if(dirs_coronal_data_decompress[dirs_idx][0] == str(1)):
            for idx in range(len(LGG_testing_idx)):
                if(dirs_coronal_data_decompress[dirs_idx][1]==str(LGG_testing_idx[idx])):
                    dirs_coronal_data_testing.append(dirs_coronal_data_decompress[dirs_idx][2])
                    dirs_coronal_label_testing.append(dirs_coronal_label_decompress[dirs_idx][2])

            for idx in range(len(LGG_validation_idx)):
                if(dirs_coronal_data_decompress[dirs_idx][1]==str(LGG_validation_idx[idx])):
                    dirs_coronal_data_validation.append(dirs_coronal_data_decompress[dirs_idx][2])
                    dirs_coronal_label_validation.append(dirs_coronal_label_decompress[dirs_idx][2])

            for idx in range(len(LGG_training_idx)):
                if(dirs_coronal_data_decompress[dirs_idx][1]==str(LGG_training_idx[idx])):
                    dirs_coronal_data_training.append(dirs_coronal_data_decompress[dirs_idx][2])
                    dirs_coronal_label_training.append(dirs_coronal_label_decompress[dirs_idx][2])
                                  
    print(len(dirs_coronal_data_decompress))
    print(len(dirs_coronal_data_testing))
    print(len(dirs_coronal_data_validation))
    print(len(dirs_coronal_data_training))

In [ ]:
import os
import numpy as np

if(False):
    for i in range(len(dirs_axial_data_training)):
        print(i)
        dummy_data = np.load(dirs_axial_data_training[i])
        dummy_data_decompress = dummy_data['arr_0']
        dummy_label = np.load(dirs_axial_label_training[i])
        dummy_label_decompress = dummy_label['arr_0']
        print(dummy_data_decompress.shape)
        plt.figure(figsize=(15,8))
        plt.subplot(2, 4, 1)
        plt.imshow(dummy_data_decompress[:,:,0],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 2)
        plt.imshow(dummy_data_decompress[:,:,1],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 3)
        plt.imshow(dummy_data_decompress[:,:,2],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 4)
        plt.imshow(dummy_data_decompress[:,:,3],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 5)
        plt.imshow(dummy_label_decompress[:,:,0],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 6)
        plt.imshow(dummy_label_decompress[:,:,1],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 7)
        plt.imshow(dummy_label_decompress[:,:,2],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 8)
        plt.imshow(dummy_label_decompress[:,:,3],cmap='gray') # BGR->RGB for plotting
        plt.show()
        

if(False):
    for i in range(len(dirs_sagittal_data_training)):
        print(i)
        dummy_data = np.load(dirs_sagittal_data_training[i])
        dummy_data_decompress = dummy_data['arr_0']
        dummy_label = np.load(dirs_sagittal_label_training[i])
        dummy_label_decompress = dummy_label['arr_0']
        print(dummy_data_decompress.shape)
        plt.figure(figsize=(15,8))
        plt.subplot(2, 4, 1)
        plt.imshow(dummy_data_decompress[:,:,0],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 2)
        plt.imshow(dummy_data_decompress[:,:,1],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 3)
        plt.imshow(dummy_data_decompress[:,:,2],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 4)
        plt.imshow(dummy_data_decompress[:,:,3],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 5)
        plt.imshow(dummy_label_decompress[:,:,0],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 6)
        plt.imshow(dummy_label_decompress[:,:,1],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 7)
        plt.imshow(dummy_label_decompress[:,:,2],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 8)
        plt.imshow(dummy_label_decompress[:,:,3],cmap='gray') # BGR->RGB for plotting
        plt.show()

if(False):
    for i in range(len(dirs_coronal_data_training)):
        print(i)
        dummy_data = np.load(dirs_coronal_data_training[i,])
        dummy_data_decompress = dummy_data['arr_0']
        dummy_label = np.load(dirs_coronal_label_training[i,])
        dummy_label_decompress = dummy_label['arr_0']
        print(dummy_data_decompress.shape)
        plt.figure(figsize=(15,8))
        plt.subplot(2, 4, 1)
        plt.imshow(dummy_data_decompress[:,:,0],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 2)
        plt.imshow(dummy_data_decompress[:,:,1],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 3)
        plt.imshow(dummy_data_decompress[:,:,2],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 4)
        plt.imshow(dummy_data_decompress[:,:,3],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 5)
        plt.imshow(dummy_label_decompress[:,:,0],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 6)
        plt.imshow(dummy_label_decompress[:,:,1],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 7)
        plt.imshow(dummy_label_decompress[:,:,2],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 8)
        plt.imshow(dummy_label_decompress[:,:,3],cmap='gray') # BGR->RGB for plotting
        plt.show()


# Create a Keras data generator

In [ ]:
import matplotlib.image as mpimg

dirs_all = os.listdir(TRAIN_PATH)
dirs_all.sort()
dirs_train = dirs_all[:2]
training_data = []
label_data = []


import keras
import os
import sys
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from time import time

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, CSVLogger
from keras.optimizers import Adam, SGD
from keras import backend as K

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(240,240,4),dim_lab=(240,240,1), n_channels=4,
                 n_classes=2, shuffle=False):
        'Initialization'
        self.dim = dim
        self.dim_lab = dim_lab        
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle

        #self.image_size = image_size #Thats new
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        #print(indexes)
        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size, *self.dim_lab))
        #print(X.shape)
        #print(y.shape)
        # Generate data
        
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            #X[i,] = np.load(STORE_PATH + "/" + ID)
            #print(ID)
            ID_val = np.load(ID)
            X[i,] = ID_val['arr_0']
            LABEL_ID = ID.replace("Data", "Label")
            # Store class
            #y[i,] = np.load(LABEL_PATH + "/" + ID)
            LABEL_ID_val = np.load(LABEL_ID)
            y[i,] = LABEL_ID_val['arr_0']

        return X, y

In [ ]:
dirs_all = [];
train=[];
enable_original_axial = True
enable_original_sagittal = True
enable_original_coronal = True


# Parameters
params = {'dim': (240,240,4),
          'dim_lab': (240,240,4),
          'batch_size': 48,
          'n_classes': 4,
          'n_channels': 4,
          'shuffle': True}



train = np.array(dirs_axial_data_training)
print(train.shape)
if(enable_original_sagittal):    
    train=np.concatenate((train,np.array(dirs_sagittal_data_training)),axis=0)
print(train.shape)
if(enable_original_coronal): 
    train=np.concatenate((train,np.array(dirs_coronal_data_training)),axis=0)
validation = np.array(dirs_axial_data_validation)
print(train.shape)
print(validation.shape)

test = np.array(dirs_axial_data_testing)
# Generators
#np.random.shuffle(validation)
#np.random.shuffle(train)


training_generator = DataGenerator(train, train, **params)

#print(training_generator)
validation_generator = DataGenerator(validation, validation, **params)

testing_generator = DataGenerator(test, test, **params)


# Show some samples of generated signal:

In [ ]:
if(False):
    x,y = training_generator.__getitem__(0);

    print(x.shape)
    print(y.shape)
    num = 0
    for num in range(32):
        plt.figure(figsize=(15,8))
        plt.subplot(2, 4, 1)
        plt.imshow(x[num,:,:,0],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 2)
        plt.imshow(x[num,:,:,1],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 3)
        plt.imshow(x[num,:,:,2],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 4)
        plt.imshow(x[num,:,:,3],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 5)
        plt.imshow(y[num,:,:,0],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 6)
        plt.imshow(y[num,:,:,1],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 7)
        plt.imshow(y[num,:,:,2],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 8)
        plt.imshow(y[num,:,:,3],cmap='gray') # BGR->RGB for plotting
        plt.show()


# Building the Deep Learning Model:

In [ ]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import backend as K
from keras.callbacks import ModelCheckpoint

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_multi(y_true, y_pred, smooth=1.0):
    ''' Dice Coefficient
    Args:
        y_true (np.array): Ground Truth Heatmap (Label)
        y_pred (np.array): Prediction Heatmap
    '''

    class_num = 4
    for i in range(class_num):
        y_true_f = K.flatten(y_true[:,:,:,i])
        y_pred_f = K.flatten(y_pred[:,:,:,i])
        intersection = K.sum(y_true_f * y_pred_f)
        loss = ((2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth))
        if i == 0:
            total_loss = loss
        else:
            total_loss = total_loss + loss
    total_loss = total_loss / class_num
    return total_loss

def dice_coef_whole_tumor(y_true, y_pred, smooth=1.0):
    ''' Dice Coefficient
    Args:
        y_true (np.array): Ground Truth Heatmap (Label)
        y_pred (np.array): Prediction Heatmap
    '''
    y_true_f = y_true[:,:,:,1] + y_true[:,:,:,2] + y_true[:,:,:,3]
    y_pred_f = y_pred[:,:,:,1] + y_pred[:,:,:,2] + y_pred[:,:,:,3]

    y_true_f = K.flatten(y_true_f)
    y_pred_f = K.flatten(y_pred_f)
    intersection = K.sum(y_true_f * y_pred_f)
    loss = ((2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth))

    return loss

def dice_coef_tumor_core(y_true, y_pred, smooth=1.0):
    ''' Dice Coefficient
    Args:
        y_true (np.array): Ground Truth Heatmap (Label)
        y_pred (np.array): Prediction Heatmap
    '''

    y_true_f = y_true[:,:,:,1] + y_true[:,:,:,3] 
    y_pred_f = y_pred[:,:,:,1] + y_pred[:,:,:,3] 

    y_true_f = K.flatten(y_true_f)
    y_pred_f = K.flatten(y_pred_f)
    intersection = K.sum(y_true_f * y_pred_f)
    loss = ((2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth))

    return loss


def dice_coef_enhancing_tumor(y_true, y_pred, smooth=1.0):
    ''' Dice Coefficient
    Args:
        y_true (np.array): Ground Truth Heatmap (Label)
        y_pred (np.array): Prediction Heatmap
    '''

    y_true_f = y_true[:,:,:,3] 
    y_pred_f = y_pred[:,:,:,3] 

    y_true_f = K.flatten(y_true_f)
    y_pred_f = K.flatten(y_pred_f)
    intersection = K.sum(y_true_f * y_pred_f)
    loss = ((2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth))

    return loss


def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef_multi(y_true, y_pred)

def UNet_multiclass():
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(conv1)
    conv1 =  BatchNormalization(axis=3)(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(conv2)
    conv2 =  BatchNormalization(axis=3)(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(conv3)
    conv3 =  BatchNormalization(axis=3)(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(conv4)
    conv4 =  BatchNormalization(axis=3)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(conv9)

    conv10 = Conv2D(4, (1, 1), activation='softmax', padding = 'same')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])
    
    return model

def UNet_multiclass_regular():
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(conv1)
    conv1 =  BatchNormalization(axis=3)(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    pool1 = Dropout(0.25)(pool1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(conv2)
    conv2 =  BatchNormalization(axis=3)(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    pool2 = Dropout(0.5)(pool2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(conv3)
    conv3 =  BatchNormalization(axis=3)(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    pool3 = Dropout(0.5)(pool3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(conv4)
    conv4 =  BatchNormalization(axis=3)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    pool4 = Dropout(0.5)(pool4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    up6 = Dropout(0.5)(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    up7 = Dropout(0.5)(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    up8 = Dropout(0.5)(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    up9 = Dropout(0.5)(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same',kernel_initializer=initializers.random_normal(stddev=0.01))(conv9)

    conv10 = Conv2D(4, (1, 1), activation='softmax', padding = 'same')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])
    
    return model

In [ ]:
model = UNet_multiclass()
model.compile(optimizer=Adam(lr=1e-4), loss=dice_coef_loss, metrics=[dice_coef_multi , dice_coef_whole_tumor, dice_coef_tumor_core , dice_coef_enhancing_tumor])
model.summary()

In [ ]:
"""
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
csv_logger = CSVLogger('my_model.csv')

earlystopper = EarlyStopping(patience=20, verbose=1)
checkpointer = ModelCheckpoint('model_BRATS12.h5', verbose=1, save_best_only=True)

history = model.fit_generator(generator=training_generator,validation_data=validation_generator, epochs=5, 
                    callbacks=[earlystopper, checkpointer, tensorboard, csv_logger])
folder_directory = os.path.join('../working/BraTs_results')
if not os.path.exists(folder_directory):
    os.makedirs(folder_directory)
    model.save('../working/BraTs_results/my_model.h5')
    model.save_weights('../working/BraTs_results/my_checkpoint')
else:
    model.save('../working/BraTs_results/my_model.h5')
    model.save_weights('../working/BraTs_results/my_checkpoint')
""" 

In [ ]:
#folder_directory = os.path.join('../kaggle/BraTs_results')
#if not os.path.exists(folder_directory):
#        os.makedirs(folder_directory)
weight_path_prev = '/kaggle/input/brats-model-weights/model_BRATS13.h5'
weight_path_last = '../working/model_BRATS13.h5'
weight_path_best = '../working/model_BRATS13_best.h5'

earlystopper = EarlyStopping(patience=20, verbose=1)
checkpointer = ModelCheckpoint(weight_path_best, verbose=1, save_best_only=True)#this saves the best weights with its architecture
if not os.path.isfile(weight_path_prev): #if the weight doesn't exist, start from scratch
    print("Starting training from the scratch:")
    csv_path = '../working/my_model13.csv'
    tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
    csv_logger = CSVLogger(csv_path)

    earlystopper = EarlyStopping(patience=20, verbose=1)
    checkpointer = ModelCheckpoint(weight_path_best, verbose=1, save_best_only=True)#this saves the best weights with its architecture
    
    history = model.fit_generator(generator=training_generator,validation_data=validation_generator, epochs=15, 
                        callbacks=[earlystopper, checkpointer, tensorboard, csv_logger])
    
    model.save(weight_path_last) #this saves the last weights with its architecture
    model.save_weights('../working/my_checkpoint')

else: #otherwise load the model and continue training
    from shutil import copyfile
    print("Starting from the perevious weights:")
    csv_path = '../working/my_model13.csv'
    copyfile('/kaggle/input/brats-model-weights/my_model13.csv' , csv_path)
    tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
    csv_logger = CSVLogger(csv_path)

    earlystopper = EarlyStopping(patience=20, verbose=1)
    checkpointer = ModelCheckpoint(weight_path_best, verbose=1, save_best_only=True)#this saves the best weights with its architecture

    model = load_model(weight_path_prev,custom_objects={'dice_coef_loss':
dice_coef_loss,'dice_coef_multi': dice_coef_multi,'dice_coef_whole_tumor': dice_coef_whole_tumor,'dice_coef_tumor_core': dice_coef_tumor_core,'dice_coef_enhancing_tumor': dice_coef_enhancing_tumor})
    history = model.fit_generator(generator=training_generator,validation_data=validation_generator, epochs=15, 
                        callbacks=[earlystopper, checkpointer, tensorboard, csv_logger])

    model.save(weight_path_last) #this saves the last weights with its architecture
    model.save_weights('../working/my_checkpoint')

In [ ]:
os.path.isfile('/kaggle/input/brats-model-weights/model_BRATS12.h5')


In [ ]:

# summarize history for accuracy
plt.plot(history.history['dice_coef_multi'])
plt.plot(history.history['val_dice_coef_multi'])
plt.title('model dice_coef_multi accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# summarize history for accuracy
plt.plot(history.history['dice_coef_whole_tumor'])
plt.plot(history.history['val_dice_coef_whole_tumor'])
plt.title('model dice_coef_whole_tumor accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# summarize history for accuracy
plt.plot(history.history['dice_coef_tumor_core'])
plt.plot(history.history['val_dice_coef_tumor_core'])
plt.title('model dice_coef_tumor_core accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# summarize history for accuracy
plt.plot(history.history['dice_coef_enhancing_tumor'])
plt.plot(history.history['val_dice_coef_enhancing_tumor'])
plt.title('model dice_coef_enhancing_tumor accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()


In [ ]:
trained_model = load_model('../working/model_BRATS13.h5', custom_objects={'dice_coef_loss':
dice_coef_loss,'dice_coef_multi': dice_coef_multi,'dice_coef_whole_tumor': dice_coef_whole_tumor,'dice_coef_tumor_core': dice_coef_tumor_core,'dice_coef_enhancing_tumor': dice_coef_enhancing_tumor})
x,y = validation_generator.__getitem__(0);
test_data = x[:,:,:,:]
pred_value = trained_model.predict(test_data, verbose = 1)
for num in range(32):
    print(num)
    plt.figure(figsize=(15,8))
    plt.subplot(3, 4, 1)
    plt.imshow(pred_value[num,:,:,0],cmap='gray') # BGR->RGB for plotting
    plt.subplot(3, 4, 2)
    plt.imshow(pred_value[num,:,:,1],cmap='gray') # BGR->RGB for plotting
    plt.subplot(3, 4, 3)
    plt.imshow(pred_value[num,:,:,2],cmap='gray') # BGR->RGB for plotting
    plt.subplot(3, 4, 4)
    plt.imshow(pred_value[num,:,:,3],cmap='gray') # BGR->RGB for plotting
    plt.subplot(3, 4, 5)
    plt.imshow(y[num,:,:,0],cmap='gray') # BGR->RGB for plotting
    plt.subplot(3, 4, 6)
    plt.imshow(y[num,:,:,1],cmap='gray') # BGR->RGB for plotting
    plt.subplot(3, 4, 7)
    plt.imshow(y[num,:,:,2],cmap='gray') # BGR->RGB for plotting
    plt.subplot(3, 4, 8)
    plt.imshow(y[num,:,:,3],cmap='gray') # BGR->RGB for plotting
    plt.subplot(3, 4, 9)
    plt.imshow(x[num,:,:,0],cmap='gray') # BGR->RGB for plotting
    plt.subplot(3, 4, 10)
    plt.imshow(x[num,:,:,1],cmap='gray') # BGR->RGB for plotting
    plt.subplot(3, 4, 11)
    plt.imshow(x[num,:,:,2],cmap='gray') # BGR->RGB for plotting
    plt.subplot(3, 4, 12)
    plt.imshow(x[num,:,:,3],cmap='gray') # BGR->RGB for plotting
    plt.show()

In [ ]:
import matplotlib.image as mpimg
from scipy.stats import skew
from scipy.stats import kurtosis
import numpy.ma as ma

trained_model = load_model('../working/model_BRATS13.h5', custom_objects={'dice_coef_loss':
dice_coef_loss,'dice_coef_multi': dice_coef_multi,'dice_coef_whole_tumor': dice_coef_whole_tumor,'dice_coef_tumor_core': dice_coef_tumor_core
                                                                        ,'dice_coef_enhancing_tumor': dice_coef_enhancing_tumor})

test_patients = ['Brats18_2013_22_1','Brats18_CBICA_ABN_1','Brats18_TCIA01_201_1','Brats18_TCIA08_406_1'
                ,'Brats18_CBICA_ARZ_1','Brats18_TCIA01_131_1','Brats18_2013_7_1','Brats18_2013_0_1'
                ,'Brats18_2013_6_1','Brats18_TCIA10_266_1']
count = 0
dice_test = 0
for nf, folder in enumerate(dirs_train):
  print("Processing Images...")
  HLG_PATH = os.path.join(TRAIN_PATH,folder)
  hlg_all = os.listdir(HLG_PATH)  

  for n, file in enumerate(hlg_all):
    if file in test_patients:
        count = count + 1
        print(file)
        print("Image: "+ str(nf) + ", " + str(n))
        flair = nib.load(os.path.join(HLG_PATH, file, file + '_flair.nii.gz')).get_data()
        t1 = nib.load(os.path.join(HLG_PATH, file, file + '_t1.nii.gz')).get_data()
        t1ce = nib.load(os.path.join(HLG_PATH, file, file + '_t1ce.nii.gz')).get_data()
        t2 = nib.load(os.path.join(HLG_PATH, file, file + '_t2.nii.gz')).get_data()
        label = nib.load(os.path.join(HLG_PATH, file, file + '_seg.nii.gz')).get_data()

        print("Axial Data:")    
        axial = 0;
        for i in range(155):
            if(np.mean(flair[:,:,i]+t1[:,:,i]+t1ce[:,:,i]+t2[:,:,i])!=0):
                data_patch_axial = np.zeros([240,240,4],dtype='float32')
                data_patch_axial[:,:,0] = normalize_image(flair[:,:,i])
                data_patch_axial[:,:,1] = normalize_image(t1[:,:,i])
                data_patch_axial[:,:,2] = normalize_image(t1ce[:,:,i])
                data_patch_axial[:,:,3] = normalize_image(t2[:,:,i])
                if (axial == 0):
                    data_patch_axial_test = np.expand_dims(data_patch_axial,axis=0)
                else:
                    data_patch_axial = np.expand_dims(data_patch_axial,axis=0)
                    data_patch_axial_test = np.concatenate((data_patch_axial_test,data_patch_axial),axis = 0)
                axial = axial + 1
                
        axial = 0
        for i in range(155):
            if(np.mean(flair[:,:,i]+t1[:,:,i]+t1ce[:,:,i]+t2[:,:,i])!=0):
                label_patch_axial = np.zeros([240,240,4],dtype='int')
                label_patch_axial[:,:,0] = (label[:,:,i]==0).astype(int)
                label_patch_axial[:,:,1] = (label[:,:,i]==1).astype(int)
                label_patch_axial[:,:,2] = (label[:,:,i]==2).astype(int)
                label_patch_axial[:,:,3] = (label[:,:,i]==4).astype(int)
                if (axial == 0):
                    label_patch_axial_test = np.expand_dims(label_patch_axial,axis=0)
                else:
                    label_patch_axial = np.expand_dims(label_patch_axial,axis=0)
                    label_patch_axial_test=np.concatenate((label_patch_axial_test,label_patch_axial),axis = 0)
                axial = axial + 1
         
        print(label_patch_axial_test.shape[0])
        print(data_patch_axial_test.shape)
        pred_value = trained_model.predict(data_patch_axial_test, verbose = 1)
        evaluation = trained_model.evaluate(x=data_patch_axial_test, y=label_patch_axial_test,batch_size = label_patch_axial_test.shape[0]
                                               )
        print(evaluation)
        dice_test = np.asarray(evaluation) +dice_test

print("test accuracy:")
print(dice_test/count)